In [ ]:
import string
import re
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import datetime
from text_unidecode import unidecode
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfTransformer, ENGLISH_STOP_WORDS, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import  metrics
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from imblearn.under_sampling import RandomUnderSampler
nltk.download('stopwords')

In [ ]:
from nltk.stem import WordNetLemmatizer 


def text_preprocessing(df, lis):
    text_data = df["MainText"]
    
    punct = set(string.punctuation)
    stopwords = set(ENGLISH_STOP_WORDS)
    lm = WordNetLemmatizer()
    
    for i in range(len(text_data)):
        line = text_data[i]
        
        line = line.strip()
        line = " ".join(line.split())

        line = unidecode(line)
        
        line = line.lower()
        line = ''.join(char for char in line if char not in punct)
        tokens = re.split('\W+', line)
        
        line = [lm.lemmatize(word) for word in tokens if word not in stopwords]
        
        lis.append(line)

In [ ]:
df = pd.read_csv("our_contest_train_2.csv")
primary = df[["PrimarySubject"]]

In [ ]:
df.dropna(subset=["MainText"], inplace=True)
df = df[["MainText", "Target"]]
tf= df.copy()
del df
df = tf.copy()
del tf
df = df.reset_index()

In [ ]:
lis = []
text_preprocessing(df, lis)
y = df[["Target"]]
del df

vectorizer = CountVectorizer()
sentence = [''.join(o) for o in lis]
del lis
vec_counter = vectorizer.fit_transform(sentence)
del sentence
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(vec_counter)

## Decision tree train

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion="gini", random_state=42)
clf = clf.fit(df, y)

## Test

In [ ]:
del X_train_tfidf
df_test = pd.read_csv("our_contest_test.csv")
df_test = df_test[["MainText"]]
primary = df_test[["PrimarySubject"]]
tf = df_test.copy()
del df_test
df_test= tf.copy()
del tf

In [ ]:
lis = []
text_preprocessing(df_test, lis)
primary = df_test[["PrimarySubject"]]

del df_test

vectorizer = CountVectorizer()
sentence = [''.join(o) for o in lis]
del lis
vec_counter = vectorizer.fit_transform(sentence)
del sentence
X_test_tfidf = tfidf_transformer.fit_transform(vec_counter)

In [ ]:
y_pred = clf.predict(X_test_tfidf)
del X_test_tfidf

In [ ]:
dummy_read = pd.read_csv('./our_contest_sample_solution_2.csv')
y_pred = pd.DataFrame(y_pred, columns=["Target"])

dummy_read['Target'] = y_pred
dummy_read['Target'] = dummy_read['Target'].astype(int)

dummy_read.to_csv("XGBoost.csv", index=False)

In [ ]:
y_pred["Target"].unique()

In [ ]:
dummy_read = pd.read_csv('./our_contest_sample_solution_2.csv')
y_pred = pd.DataFrame(y_pred, columns=["Target"])

dummy_read['Target'] = y_pred
dummy_read['Target'] = dummy_read['Target'].astype(int)

dummy_read.to_csv("Lightgbm.csv", index=False)